In [2]:
import csv
import re

def cleanAndWriteStory(story, file):
    storyNoIntro = story[1:]
    # Get rid of linespace
    storyNoIntro.pop(1)

    # get rid of actions and write to file
    for lineID, line in enumerate(storyNoIntro):
        lineStripped = re.sub("{.*}", "", line)
        lineStripped = re.sub("\[.*\]", "", lineStripped)
        newF.write(lineStripped+"\n")
    newF.write("<|endoftext|>\n")
    
# The story file is expected to be tab delimited
fromFile = "storyville"
n = 10000
createSingleFile = False
with open(f"data/{fromFile}.txt", "r", encoding="utf-8") as f:
    stories = csv.reader(f, delimiter="\t")
    
    if createSingleFile:
        with open(f"data/{fromFile}Stripped.txt", "r", encoding="utf-8") as newF:
            for storyID, story in enumerate(stories):
                cleanAndWriteStory(story, newF)
    else:
        i = -1
        for storyID, story in enumerate(stories):
            if storyID%n == 0:
                i += 1
            # appends the stories to the files. When creating new stories delete the files in the stories folder
            with open(f"data/stories/{fromFile}Stripped{i}.txt", "a", encoding="utf-8") as newF:
                cleanAndWriteStory(story, newF)

# A single stripped file ended up being 1.5GB and resulted in a memory error when calculating its BPE
# Hopefully splitting the stories into multiple files does not produce a OOM error.